## Local training 테스트

- SageMaker managed training을 하기에 앞서 local 환경에서 training을 진행해 봅니다.

In [ ]:
%store -r

In [ ]:
model_download_path

In [ ]:
import sys
import os
import torch
import transformers
from datasets import load_dataset, load_from_disk

In [ ]:
from transformers import BitsAndBytesConfig
quant_4bit = True
quant_8bit = False

if quant_4bit:
    nf4_config = BitsAndBytesConfig(
       load_in_4bit=True,
       bnb_4bit_quant_type="nf4",
       bnb_4bit_use_double_quant=True,
       bnb_4bit_compute_dtype=torch.bfloat16
    )
else:
    nf4_config = None

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
device_map = "auto"

tokenizer = AutoTokenizer.from_pretrained(model_download_path)

model = AutoModelForCausalLM.from_pretrained(
    model_download_path,
    load_in_8bit=True if quant_8bit else False,
    torch_dtype=torch.float16,
    device_map=device_map,
    #cache_dir=cache_dir,
    quantization_config=nf4_config,
)

In [ ]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
)

model = prepare_model_for_kbit_training(model)

lora_r  = 8
lora_alpha = 32
lora_dropout = 0.05
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"
  ]

config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

In [ ]:
model.print_trainable_parameters()

In [ ]:
import os
train_data = load_from_disk(os.path.join("dataset", "train"))
val_data = load_from_disk((os.path.join("dataset", "val")))

In [ ]:

num_epochs = 1
batch_size = 2

learning_rate = 3e-5
gradient_accumulation_steps = 2
val_set_size = len(val_data)
output_dir = 'output'
world_size = 1
ddp = world_size != 1
group_by_length = False

In [ ]:
bf16 = True if torch.cuda.get_device_capability()[0] == 8 else False

In [ ]:
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    warmup_steps=100,
    num_train_epochs=num_epochs,
    learning_rate=learning_rate,
    bf16=bf16,  # g4dn (Nvidia T4) cannot use bf16
    logging_steps=2,
    optim="paged_adamw_8bit",
    evaluation_strategy="steps" if val_set_size > 0 else "no",
    save_strategy="steps",
    eval_steps=200 if val_set_size > 0 else None,
    save_steps=30,
    output_dir=output_dir,
    load_best_model_at_end=True if val_set_size > 0 else False,
    ddp_find_unused_parameters=False if ddp else None,
    report_to="none",
    group_by_length=group_by_length,
)


In [ ]:

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=train_args,
    data_collator=transformers.DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

In [ ]:
model.config.use_cache = False

# old_state_dict = model.state_dict
# model.state_dict = (lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())).__get__(
#     model, type(model)
# )

if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

train_result = trainer.train()

In [ ]:
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
#trainer.save_metrics("train", metrics)

In [ ]:
trainer.model.save_pretrained(output_dir)

In [ ]:
# Free memory for merging weights
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16
)

In [ ]:
merged_model = model.merge_and_unload()

In [ ]:
merged_path = "merged_model"
os.makedirs(merged_path, exist_ok=True)
merged_model.save_pretrained(merged_path, safe_serialization=True)

In [ ]:
%store merged_path